In [1]:
from vibes import *

In [261]:
P = construct_P(4, 3, 2); P

         Current function value: 0.000000
         Iterations: 124
         Function evaluations: 182
         Gradient evaluations: 177


/Users/heyredhat/opt/anaconda3/envs/spring2025/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


array([[0.5 , 0.25, 0.25, 0.  ],
       [0.25, 0.5 , 0.  , 0.25],
       [0.25, 0.  , 0.5 , 0.25],
       [0.  , 0.25, 0.25, 0.5 ]])

In [262]:
P_data = extract_P_data(P)
n, alpha, beta, Phi = P_data["n"], P_data["alpha"], P_data["beta"], P_data["Phi"]
eta = -beta/(alpha+1)
A = np.array([[[eta*(delta(i,j) - delta(i,k) - delta(j,k)) for k in range(n)] for j in range(n)] for i in range(n)])
V = np.array([[[delta(i,j)*delta(i,k) for k in range(n)] for j in range(n)] for i in range(n)])

In [268]:
@jax.jit
def obj(X):
    p = X + 1/n
    p = p/jp.sum(p)
    C = P @ Phi @ jp.einsum("ijk, k", V-A, p) @ P @ Phi
    p2 = C @ Phi @ p
    p2 = p2/jp.sum(p2)
    return jp.linalg.norm(p - p2) 

@jax.jit
def obj2(X):
    p = X + 1/n
    p = p/jp.sum(p)
    C = P @ Phi @ jp.einsum("ijk, k", V-A, p) @ P @ Phi
    return jp.linalg.eigvals(C).real

result = sc.optimize.minimize(obj, np.random.randn(n),\
                                jac=jax.jit(jax.jacrev(obj)),\
                                constraints=[{"type": "ineq", "fun": obj2, "jac":jax.jit(jax.jacrev(obj2))}],\
                                tol=1e-16, options={"disp": True, "maxiter": 10000})
p = result.x + 1/n
p = p/jp.sum(p)
p

Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1.97223636870937e-11
            Iterations: 47
            Function evaluations: 145
            Gradient evaluations: 47


Array([0.231, 0.499, 0.001, 0.269], dtype=float64)

In [269]:
C = C_constructor(P, A=A)
Cp = C(p); Cp

Array([[ 0.154,  0.16 , -0.006, -0.   ],
       [ 0.16 ,  0.333,  0.   ,  0.173],
       [-0.006,  0.   ,  0.   ,  0.007],
       [-0.   ,  0.173,  0.007,  0.179]], dtype=float64)

In [270]:
p2 = (Cp @ Phi @ p)
p2_ = p2/jp.sum(p2)
jp.linalg.norm(p - p2_) 

Array(0., dtype=float64)

In [271]:
np.linalg.eigvals(Cp @ Phi)

array([ 0.667,  0.333, -0.   , -0.   ])